In [12]:
%load_ext autoreload
%autoreload 2
%run ./initializer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [13]:
import polars as pl
from lgbm_classifier import LGBMClassifierModel
from lightgbm import LGBMClassifier

In [14]:
train_df = pl.read_parquet("data/training_set_processed.parquet")
test_df = pl.read_parquet("data/test_set_processed.parquet")

In [15]:
lgbm_class_model = LGBMClassifierModel(
    df = train_df,              # Only initialized to get column names
    objective="binary",         # binary click prediction
    boosting_type="gbdt",       # gradient‑boosted decision trees
    n_estimators=1000,          # upper cap on number of trees
    learning_rate=0.05,
)
X_train, y_train, X_val, y_val, val_idx = lgbm_class_model.format_data(train_df)

In [16]:
lgbm_class_model.fit(
    X_train,
    y_train,
    X_val,
    y_val,
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 177546, number of negative: 3789131
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.283486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11309
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 92
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.044759 -> initscore=-3.060662
[LightGBM] [Info] Start training from score -3.060662
Training until validation scores don't improve for 50 rounds
[20]	valid_0's auc: 0.709532	valid_0's binary_logloss: 0.171343
[40]	valid_0's auc: 0.718033	valid_0's binary_logloss: 0.168817
[60]	valid_0's a

In [18]:
lgbm_class_model.evaluate_validation(
    X_val,
    y_val,
    train_df,
    val_idx
)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Validation AUC: 0.7426781732230038


/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Validation NDCG@5: 0.37621275624795153


(0.7426781732230038, 0.37621275624795153)

In [19]:
lgbm_class_model.get_ndcg_score(train_df)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Validation NDCG@5: 0.4455873831934228


0.4455873831934228

In [20]:
lgbm_class_model.get_final_predictions(test_df)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


srch_id,prop_id
i32,i32
1,54937
1,28181
1,99484
1,61934
1,5543
…,…
332787,29018
332787,32019
332787,99509
